In [ ]:
from elasticsearch import Elasticsearch, helpers
from pathlib import Path
import time
import gcomc_reader

es = Elasticsearch("http://es:9200")
product = 'LST'
source_path = "../source"

In [ ]:
product_index = product.lower()
# es.indices.delete(product_index) #indexを作り直す場合 

if not es.indices.exists(index=product_index):
    es.indices.create(index=product_index, body={
        "mappings": {
            "properties": {
                "granule_id": {
                  "type": "keyword"
                },
                "location": {
                  "type": "geo_point"
                },
                "value": {
                    "type": "float"
                },
                "observation_datetime": {
                  "type": "date"
                },
                "qa_flags": {
                    "type": "short"
                }
            }
        }
    })

In [ ]:
process_index = 'process'
# es.indices.delete(process_index) #indexを作り直す場合 

if not es.indices.exists(index=process_index):
    es.indices.create(index=process_index, body={
        "mappings": {
            "properties": {
                "location": {
                  "type": "geo_shape"
                },
                "start_datetime": {
                  "type": "date"
                },
                "end_datetime": {
                  "type": "date"
                },
                "processed_datetime": {
                  "type": "date"
                },
                "good": {
                    "type": "boolean"
                },
                "algorithm_version": {
                    "type": "keyword"
                },
                "product": {
                    "type": "keyword"
                },
                "done": {
                    "type": "boolean"
                }
            }
        }
    })

In [ ]:
p = Path(source_path)

for file_path in p.glob("*{}*.h5".format(product)):
    r = gcomc_reader.Tile(str(file_path))
    
    processed = es.search(index=process_index, body={"query":{"bool":{"must":[{"term":{"_id":r.granule_id}},{"term":{"done":True}}]}}})
    if processed['hits']['total']['value'] == 0 and r.resolution == 'Q':
        print("start: {}".format(r.granule_id))
        start = time.time()
        es.index(index=process_index, id=r.granule_id, body={
            "location": {
                "type" : "polygon",
                "coordinates" : [
                  [r.lower_left, r.lower_right, r.upper_right, r.upper_left, r.lower_left]
                ]
            },
            "start_datetime": r.start_time,
            "end_datetime": r.end_time,
            "processed_datetime": r.processed,
            "good": r.is_good(),
            "algorithm_version": r.algorithm_version,
            "product": r.product,
            "done": False
        })

        if r.is_good() == True:
            items = []
            for line in range(r.lines):
                for pixel in range(r.pixels):
                    try:
                        item = r.get_point(line, pixel)
                        if item['value'] is not None and item['observation_datetime'] is not None:
                            item['granule_id'] = r.granule_id
                            id = '{}_{}_{}'.format(r.granule_id, line, pixel)
                            items.append({'_op_type': 'index', '_index':product_index, '_id':id, '_source':item})

                            if len(items)>10000:
                                helpers.bulk(es, items)
                                items = []
                    except:
                        pass

            if len(items)>0:
                helpers.bulk(es, items)
        es.update(index=process_index, id=r.granule_id, body={"doc": {"done": True}})
        print("done: {}[s]".format(time.time() - start))
    else:
        print("passed: {}".format(r.granule_id))

In [ ]:
print("created points: {}".format(es.count(index=product_index)))
print("processed files: {}".format(es.count(index=process_index)))